# SCC-DFTB repulsive potential generation (2-body)

Input: DFT_DB.db DFTB_DB.db 

Output: CCS parameters to be used in conjunction with the SK-tables generated in gen_SK.ipynb and CCS_DB.db 



In [ ]:
PARAUTOMATIK_PATH='WRITE-Your-PATH-PARAUTOMATIK-Here'   # Add your installation path here

In [ ]:
import os, sys, warnings
sys.path.append(PARAUTOMATIK_PATH+'/pgm/CCS/')
sys.path.append(PARAUTOMATIK_PATH+'/pgm/python_scripts')
import json
import itertools as it
from collections import OrderedDict, defaultdict
import numpy as np
from ase import Atoms
from ase import io
from ase.calculators.neighborlist import NeighborList
import ase.db as db
from ccs.common.exceptions import ScriptError 
from ccs.fitting.main import twp_fit
from ccs.ase_tools.ccs_ase_calculator import CCS
from rep_parautomatik import pair_dist, generate_ccsdata, generate_ccsdb, write_input, fit_ccs
base_dir=os.getcwd()
os.chdir(base_dir)
print('Base dir is:', base_dir)
warnings.filterwarnings('ignore')


In [ ]:
#generate data generates trainingset in the format for CCS (structures.json)
cwd = os.getcwd()
if not os.path.isdir(cwd+"/CCS/"):
    os.mkdir(cwd+"/CCS/")

os.chdir(cwd+"/CCS/")
DFTB_DB=db.connect(base_dir+"/DFTB_DB.db")   
DFT_DB=db.connect(base_dir+"/DFT_DB.db")   
generate_ccsdata(DFT_DB,DFTB_DB)
os.chdir(base_dir) 

In [ ]:
#input for fitting data using CCS 

os.chdir(base_dir+'/CCS')            
input = {
"Reference":"structures.json",
"General":{
	"interface":"DFTB",
}
}

write_input(input)
os.chdir(base_dir) 

In [ ]:
os.chdir(base_dir+"/CCS/")
class Args:
    cmdlineargs=None
    input = 'input.json'

args=Args()
fit_ccs(args)
os.chdir(base_dir) 

# Analysis
![title](CCS/CCS_fitting_summary.png)

In [7]:
os.chdir(base_dir)

struct=db.connect("DFTB_DB.db")
db_ccs=db.connect("CCS_DB.db")

os.chdir(base_dir+"/CCS/")
with open('CCS_params.json', 'r') as f:
    CCS_params = json.load(f)

for row in struct.select():  
    structure=row.toatoms()
    structure.calc=CCS(CCS_params=CCS_params)
    structure.get_potential_energy()
    structure.get_forces()
    db_ccs.write(structure,key=row.key,CCS=True)

os.chdir(base_dir)